This notebook is an exercise in using the [**Cairo**](http://cairographics.org/) 2D graphics library via the [**Qahirah**](https://github.com/ldo/qahirah) high-level binding to construct interactive geometrical diagrams.

In this case, the exercise is to derive the relationship between a solid angle on the surface of a sphere and the vertex angle of the subtending cone.

In [ ]:
from IPython.html.widgets import \
    interact
from IPython.display import \
    display as ipython_display, \
    Latex, \
    display_png
import math
import qahirah as qah
from qahirah import \
    CAIRO, \
    Colour, \
    Matrix, \
    Path, \
    Vector

pix = qah.ImageSurface.create \
  (
    format = CAIRO.FORMAT_RGB24,
    dimensions = (400, 400)
  )
ctx = None

def reset() :
    "(re)initializes the drawing context, wiping out any existing drawing."
    global ctx
    del ctx
    ctx = qah.Context.create(pix)
    (ctx
       .save()
       .set_source_colour(Colour.grey(.95))
       .paint()
       .restore()
       .select_font_face(family = "serif", slant = CAIRO.FONT_SLANT_ITALIC, weight = CAIRO.FONT_WEIGHT_NORMAL)
       .set_font_size(24)
    )
#end reset

def display() :
    "(re)displays what has been drawn."
    display_png(pix.to_png_bytes(), raw = True)
#end display

def display_latex(s) :
    "formats and displays a string as Latex markup."
    ipython_display(Latex(s))
#end display_latex

reset()

In two dimensions, the *angle subtended by an arc* can be defined as the ratio of the arc length to the radius:

In [ ]:
@interact(angle = (0.1, 1.5, 0.1))
def demo_arc_angle(angle) :
    display_latex("$$s = r \\theta$$")
    reset()
    radius = abs(pix.dimensions) * 0.5
    small_radius = radius * 0.1
    text_mid_y = (ctx.font_extents.ascent - ctx.font_extents.descent) / 2
    (ctx
        .translate(Vector(0.1, 0.5) * pix.dimensions)
        .set_line_width(2)
        .set_source_colour(Colour.grey(0))
        .move_to((0, 0))
        .line_to(Vector(radius, 0).rotate(- angle / 2))
        .arc((0, 0), radius, - angle / 2, angle / 2, False)
        .close_path()
        .stroke()
        .move_to((0, 0))
        .line_to(Vector(small_radius, 0).rotate(- angle / 2))
        .arc((0, 0), small_radius, - angle / 2, angle / 2, False)
        .close_path()
        .stroke()
        .move_to(Vector(radius, 0).rotate(- angle * 1.1 / 2) * 0.5)
        .show_text("r")
        .move_to(Vector(small_radius * 1.3, text_mid_y))
        .show_text("\u03b8")
        .move_to(Vector(radius, 0) * 1.02 + Vector(0, text_mid_y))
        .show_text("s")
    )
    display()
#end demo_arc_angle

In the above, the radius of the circle is $r$, the length of the arc is $s$, and the angle subtended at the centre of the circle is $\theta$.

In three dimensions, an analogous definition may be applied to the *solid angle subtended by a circle* on the surface of a sphere. The radius of the sphere is $r$, and consider a cone with its apex at the centre of the sphere, cutting through its surface to form a circle. Let the area of the circular portion cut out of the sphere be $a$, and let the *half-angle* subtended by the circle at the centre of the sphere (apex of the cone) be $\theta$. Call the solid angle $\phi$. This will be defined as

$$\phi = {a \over r^2} $$

The unit for $\phi$ is called *steradians*, where that for flat angles like $\theta$ is in radians. Mathematically, both kinds of angles are expressed as ratios, and hence are dimensionless numbers.

In [ ]:
@interact(angle = (0.1, 1.5, 0.1))
def demo_sphere_angle(angle) :
    display_latex("$$a = r^2 \phi$$")
    reset()
    radius = abs(pix.dimensions) * 0.3
    foreshortened_radius = radius * 0.8
    small_radius = radius * 0.2
    text_mid_y = (ctx.font_extents.ascent - ctx.font_extents.descent) / 2
    line_colour = Colour.grey(0)
    (ctx
        .translate(Vector(0.5, 0.5) * pix.dimensions)
        .set_line_width(2)
        .set_source_colour(line_colour)
        .arc((0, 0), radius, 0, qah.circle, False)
        .close_path()
        .stroke()
        .move_to((0, 0))
        .line_to(Vector(foreshortened_radius, 0).rotate(- angle / 2))
        .move_to((0, 0))
        .line_to(Vector(foreshortened_radius, 0).rotate(angle / 2))
        .stroke()
        .move_to((0, 0))
        .line_to(Vector(foreshortened_radius, 0))
        .stroke()
        .move_to(Vector(foreshortened_radius * math.cos(angle / 2), 0))
        .line_to(Vector(foreshortened_radius, 0).rotate(angle / 2))
        .move_to((0, 0))
        .line_to(Vector(small_radius, 0))
        .arc((0, 0), small_radius, angle / 2, angle / 2, False)
        .close_path()
        .stroke()
        .append_path
          (
            Path.create_arc
              (
                centre = (0, 0),
                radius = foreshortened_radius,
                angle1 = 0,
                angle2 = qah.circle,
                negative = False
              )
                .transform
                  (
                        Matrix.translate(Vector(1, 0) * foreshortened_radius * math.cos(angle / 2))
                    *
                        Matrix.scale(math.sin(angle / 2))
                    *
                        Matrix.scale((0.5, 1))
                  )
          )
        .set_source_colour(Colour.grey(0, 0.2))
        .fill_preserve()
        .set_source_colour(line_colour)
        .stroke()
        .move_to(Vector(radius, 0).rotate(- angle * 1.1 / 2) * 0.5)
        .show_text("r")
        .move_to(Vector(foreshortened_radius, 0).rotate(angle / 4))
        .show_text("r")
        .rel_move_to((0, ctx.font_extents.descent))
        .show_text("a")
        .move_to(Vector(small_radius * 1.3, 0).rotate(angle / 2) + Vector(0, text_mid_y))
        .show_text("\u03b8")
        .move_to(Vector(foreshortened_radius, 0).rotate(- angle / 4) * 1.05 + Vector(0, text_mid_y))
        .show_text("a")
        .move_to(Vector(0, radius)* 0.5)
        .show_text("\u03d5 = ?")
    )
    display()
#end demo_sphere_angle

How do we calculate $\phi$ in terms of $\theta$?

First of all, the radius of the perimeter of the circular area is

$$r_a = r \sin \theta$$

(Note that, because of the convexity of the sphere, the centre of this perimeter circle resides *within* the sphere, not on its surface.) Now consider the incremental angle $\mathrm{d}\theta$ as $\theta$ changes. The corresponding incremental change in the area of the circle is its perimeter multiplied by the incremental change in the radius, which is

$$\begin{align}
\mathrm{d}a & = 2 \pi r_a r \mathrm{d} \theta \\
& = 2 \pi r^2 \sin \theta \mathrm{d} \theta
\end{align}$$

Integrating this gives

$$\begin{align}
a & = \int 2 \pi r^2 \sin \theta \mathrm{d} \theta \\
& = C - 2 \pi r^2 \cos \theta
\end{align}$$

for some constant $C$. But $a$ has to be zero when $\theta = 0$. Therefore,

$$C = 2 \pi r^2$$

from which

$$a = 2 \pi r^2 (1 - \cos \theta)$$

or, in other words,

$$\begin{align}
\phi & = {a \over r^2} \\
& = 2 \pi (1 - \cos \theta)
\end{align}$$

Let us check a couple of special cases: the surface area of a sphere of radius $r$ is $a = 4 \pi r^2$. Therefore the complete solid angle of a sphere is ${a \over r} = 4 \pi$ steradians. When $\theta = {\pi \over 2}$, then $cos \theta = 0$, and the above formula should give us the solid angle of a hemisphere, which is $2 \pi (1 - 0) = 2 \pi$, or half that of the sphere. Which is correct.

The formula also checks out if we set $\theta = \pi$, in which case $\cos \theta = -1$, and we get $\phi = 2 \pi (1 + 1) = 4 \pi$, the full solid angle of the sphere.